## GCN-Pytorch

In [ ]:
!git clone https://github.com/dragen1860/GCN-PyTorch.git

Cloning into 'GCN-PyTorch'...
remote: Enumerating objects: 72, done.
remote: Total 72 (delta 0), reused 0 (delta 0), pack-reused 72
Unpacking objects: 100% (72/72), done.


In [ ]:
!pwd

/content


In [ ]:
%cd GCN-PyTorch/

/content/GCN-PyTorch


In [ ]:
!python train.py --epochs 400 --dataset pubmed

Namespace(dataset='pubmed', dropout=0.5, early_stopping=10, epochs=400, hidden=16, learning_rate=0.01, max_degree=3, model='gcn', weight_decay=0.0005)
adj: (19717, 19717)
features: (19717, 500)
y: (19717, 3) (19717, 3) (19717, 3)
mask: (19717,) (19717,) (19717,)
x : tensor(indices=tensor([[    0,     0,     0,  ..., 19716, 19716, 19716],
                       [  394,   384,   382,  ...,    16,     7,     1]]),
       values=tensor([0.0283, 0.0162, 0.0125,  ..., 0.0104, 0.0030, 0.0145]),
       device='cuda:0', size=(19717, 500), nnz=988031, layout=torch.sparse_coo)
sp: tensor(indices=tensor([[    0,  1378,  1544,  ..., 19715, 16030, 19716],
                       [    0,     0,     0,  ..., 19715, 19716, 19716]]),
       values=tensor([0.1667, 0.0671, 0.0680,  ..., 0.5000, 0.2500, 0.5000]),
       device='cuda:0', size=(19717, 19717), nnz=108365, layout=torch.sparse_coo)
input dim: 500
output dim: 3
num_features_nonzero: 988031
/content/GCN-PyTorch/utils.py:45: UserWarning: indexing w

## Spektral

In [ ]:
!pip install spektral

     |████████████████████████████████| 122kB 7.2MB/s 


In [ ]:
!git clone https://github.com/danielegrattarola/spektral.git

Cloning into 'spektral'...
remote: Enumerating objects: 7359, done.
remote: Counting objects: 100% (284/284), done.
remote: Compressing objects: 100% (193/193), done.
remote: Total 7359 (delta 164), reused 160 (delta 80), pack-reused 7075
Receiving objects: 100% (7359/7359), 12.26 MiB | 11.98 MiB/s, done.
Resolving deltas: 100% (4537/4537), done.


In [ ]:
%cd spektral/examples/node_prediction/

/content/spektral/examples/node_prediction


In [ ]:
!python citation_gcn.py

2021-05-17 18:22:47.083330: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Pre-processing node features
/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
2021-05-17 18:22:53.345597: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-17 18:22:53.346798: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-05-17 18:22:53.409572: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-17 18:22:53.410284: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with proper

In [ ]:
"""
citation_gcn.py
"""
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from spektral.data.loaders import SingleLoader
from spektral.datasets.citation import Citation
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import AdjToSpTensor, LayerPreprocess

learning_rate = 1e-2  # 학습률
seed = 0
epochs = 200          # 데이터를 몇 번 사용해서 학습할 건지
patience = 10
data = "cora"

tf.random.set_seed(seed=seed)  # seed 고정

'''
Load data - datasets/citation.py
dataset name = cora
normalize true = normalize the features
transforms
transforms/layer_preprocess.py & adj_to_sp_tensor.py
LayerPreprocess(GCNConv) - preprocess 기능을 인접행렬에 적용
AdjToSpTensor()] - 인접행렬을 sparse tensor로 변환
'''
dataset = Citation(
    data, normalize_x=True, transforms=[LayerPreprocess(GCNConv), AdjToSpTensor()]
)


# 노드에 대한 avg_loss를 계산하기 위해 binary masks를 sample weights로 변환
def mask_to_weights(mask):
    return mask.astype(np.float32) / np.count_nonzero(mask)
    # mask의 0이 아닌 값의 개수


weights_tr, weights_va, weights_te = (
    mask_to_weights(mask)
    for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
)

# architecture 구현
# n_labels: number of channels in output
# n_input_channels: number of input channels, required for tf 2.1

model = GCN(n_labels=dataset.n_labels, n_input_channels=dataset.n_node_features)
model.compile(
    optimizer=Adam(learning_rate),  # 최적화 알고리즘
    loss=CategoricalCrossentropy(reduction="sum"),  # 손실함수
    weighted_metrics=["acc"], # 학습과 테스트 과정에서 sample_weight이나 class_weight로 가중치를 적용하여 평가할 측정항목의 리스트
)

# Train model
'''
single loader - 단일 그래프를 나타내는 Tensor를 생성
dataset = citation
sample_weights - 주어진 경우 출력에 추가
output -  tuple(inputs, labels) or (inputs, labels, sample_weights)
'''
loader_tr = SingleLoader(dataset, sample_weights=weights_tr)
loader_va = SingleLoader(dataset, sample_weights=weights_va)
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)

# Evaluate model
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=weights_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))


In [ ]:
# gcn.py
import tensorflow as tf

from spektral.layers.convolutional import gcn_conv


class GCN(tf.keras.Model):
    """
    This model, with its default hyperparameters, implements the architecture
    from the paper:

    > [Semi-Supervised Classification with Graph Convolutional Networks](https://arxiv.org/abs/1609.02907)<br>
    > Thomas N. Kipf and Max Welling

    **Mode**: single, disjoint, mixed, batch.

    **Input**

    - Node features of shape `([batch], n_nodes, n_node_features)`
    - Weighted adjacency matrix of shape `([batch], n_nodes, n_nodes)`

    **Output**

    - Softmax predictions with shape `([batch], n_nodes, n_labels)`.

    **Arguments**

    - `n_labels`: number of channels in output;
    - `channels`: number of channels in first GCNConv layer;
    - `activation`: activation of the first GCNConv layer;
    - `output_activation`: activation of the second GCNConv layer;
    - `use_bias`: whether to add a learnable bias to the two GCNConv layers;
    - `dropout_rate`: `rate` used in `Dropout` layers;
    - `l2_reg`: l2 regularization strength;
    - `n_input_channels`: number of input channels, required for tf 2.1;
    - `**kwargs`: passed to `Model.__init__`.
    """

    def __init__(
        self,
        n_labels,
        channels=16,
        activation="relu",  # 활성화 함수
        output_activation="softmax",
        use_bias=False,
        dropout_rate=0.5,
        l2_reg=2.5e-4,
        n_input_channels=None,
        **kwargs,
    ):
        super().__init__(**kwargs)

        self.n_labels = n_labels
        self.channels = channels
        self.activation = activation
        self.output_activation = output_activation
        self.use_bias = use_bias
        self.dropout_rate = dropout_rate
        self.l2_reg = l2_reg
        self.n_input_channels = n_input_channels
        reg = tf.keras.regularizers.l2(l2_reg)
        self._d0 = tf.keras.layers.Dropout(dropout_rate)
        self._gcn0 = gcn_conv.GCNConv(
            channels, activation=activation, kernel_regularizer=reg, use_bias=use_bias
        )
        self._d1 = tf.keras.layers.Dropout(dropout_rate)
        self._gcn1 = gcn_conv.GCNConv(
            n_labels, activation=output_activation, use_bias=use_bias
        )

        if tf.version.VERSION < "2.2":
            if n_input_channels is None:
                raise ValueError("n_input_channels required for tf < 2.2")
            x = tf.keras.Input((n_input_channels,), dtype=tf.float32)
            a = tf.keras.Input((None,), dtype=tf.float32, sparse=True)
            self._set_inputs((x, a))

    def get_config(self):
        return dict(
            n_labels=self.n_labels,
            channels=self.channels,
            activation=self.activation,
            output_activation=self.output_activation,
            use_bias=self.use_bias,
            dropout_rate=self.dropout_rate,
            l2_reg=self.l2_reg,
            n_input_channels=self.n_input_channels,
        )

    def call(self, inputs):
        if len(inputs) == 2:
            x, a = inputs
        else:
            x, a, _ = inputs  # So that the model can be used with DisjointLoader
        if self.n_input_channels is None:
            self.n_input_channels = x.shape[-1]
        else:
            assert self.n_input_channels == x.shape[-1]
        x = self._d0(x)
        x = self._gcn0([x, a])
        x = self._d1(x)
        return self._gcn1([x, a])
